IMPORTS

In [6]:
import cv2
import os

**SIMPLE GREYSCALE**

In [277]:
def display_resized_grayscale_image(image_path, width=None, height=None):
    # Check if the file exists
    if not os.path.exists(image_path):
        print("Error: File does not exist at", image_path)
        return

    # Check if the file is readable
    if not os.access(image_path, os.R_OK):
        print("Error: Cannot read file at", image_path)
        return

    # Read the image from the file path
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Unable to load image from", image_path)
        return

    # Resize the image if width or height is provided
    if width is not None and height is not None:
        image = cv2.resize(image, (width, height))

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Display the grayscale image
    cv2.imshow("Grayscale Image", gray_image)
    cv2.waitKey(0)  # Wait indefinitely until a key is pressed
    cv2.destroyAllWindows()  # Close all OpenCV windows



In [278]:
image_path = "Desktop/Train_Samples_V0/Train_Samples_V0/20240328_145148.jpg"
display_resized_grayscale_image(image_path, width=600, height=900)  # Specify width and height for resizing

**BETTER READABLE PLAIN CONTOUR**

In [283]:
def display_resized_grayscale_image_with_contours(image_path, width=None, height=None):
    # Check if the file exists
    if not os.path.exists(image_path):
        print("Error: File does not exist at", image_path)
        return

    # Check if the file is readable
    if not os.access(image_path, os.R_OK):
        print("Error: Cannot read file at", image_path)
        return

    # Read the image from the file path
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Unable to load image from", image_path)
        return

    # Resize the image if width or height is provided
    if width is not None and height is not None:
        image = cv2.resize(image, (width, height))

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply convolution for edge detection (you can use other methods too)
    edges = cv2.Canny(gray_image, 100, 200)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the original image
    image_with_contours = image.copy()
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)  # Green color for contours

    # Display the image with contours
    cv2.imshow("Image with Contours", image_with_contours)
    cv2.waitKey(0)  # Wait indefinitely until a key is pressed
    cv2.destroyAllWindows()  # Close all OpenCV windows

In [284]:
image_path = "Desktop/Train_Samples_V0/Train_Samples_V0/20240328_153456.jpg"
display_resized_grayscale_image_with_contours(image_path, width=600, height=900)  # Specify width and height for resizing

**PLAIN CONTOUR MISSES THIN LINES**

In [99]:
import numpy as np

In [255]:
def compute_threshold(image):
    # Compute MWR and AWR for each line
    mwr_values = []
    awr_values = []
    for row in image:
        # Count white run-lengths
        white_runs = [len(run) for run in np.split(row, np.where(np.diff(row) != 0)[0] + 1) if run.size > 0 and run[0] == 255]
        if white_runs:
            mwr_values.append(np.median(white_runs))
            awr_values.append(len(row) / (len(white_runs) + 1))  # +1 to avoid division by zero
    # Check if there are values in mwr_values and awr_values
    if mwr_values and awr_values:
        # Compute threshold using MWR and AWR
        threshold = np.median(mwr_values) * np.median(awr_values)
        return threshold
    else:
        print("Error: Unable to compute threshold. Image may not contain any white pixels.")
        return None

In [256]:
def extract_bounding_blocks(image, contours, threshold, padding=20):
    if threshold is None:
        print("Error: Threshold value is None. Cannot extract bounding blocks.")
        return []

    bounding_blocks = []
    for contour in contours:
        # Get bounding box of contour
        x, y, w, h = cv2.boundingRect(contour)
        # Apply padding
        x -= padding
        y -= padding
        w += 2 * padding
        h += 2 * padding
        # Check if width is greater than threshold
        if w > threshold:
            # Merge with existing bounding boxes if overlapping
            merged = False
            for i, (bx, by, bw, bh) in enumerate(bounding_blocks):
                if (x >= bx and x <= bx + bw) or (x + w >= bx and x + w <= bx + bw) or \
                   (bx >= x and bx <= x + w) or (bx + bw >= x and bx + bw <= x + w):
                    # Overlapping, merge boxes
                    x = min(x, bx)
                    y = min(y, by)
                    w = max(x + w, bx + bw) - x
                    h = max(y + h, by + bh) - y
                    bounding_blocks[i] = (x, y, w, h)
                    merged = True
                    break
            if not merged:
                bounding_blocks.append((x, y, w, h))
    return bounding_blocks

In [257]:
def display_resized_grayscale_image_with_contours_and_bounding_blocks(image_path, width=None, height=None):
    # Check if the file exists
    if not os.path.exists(image_path):
        print("Error: File does not exist at", image_path)
        return

    # Check if the file is readable
    if not os.access(image_path, os.R_OK):
        print("Error: Cannot read file at", image_path)
        return

    # Read the image from the file path
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Unable to load image from", image_path)
        return

    # Resize the image if width or height is provided
    if width is not None and height is not None:
        image = cv2.resize(image, (width, height))

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply convolution for edge detection (you can use other methods too)
    edges = cv2.Canny(gray_image, 100, 200)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Compute threshold
    threshold = compute_threshold(gray_image)

    # Extract bounding blocks based on contours and threshold
    bounding_blocks = extract_bounding_blocks(gray_image, contours, threshold)

    # Draw contours on the original image
    image_with_contours = image.copy()
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)  # Green color for contours

    # Draw bounding blocks
    for x, y, w, h in bounding_blocks:
        cv2.rectangle(image_with_contours, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color for bounding blocks

    # Display the image with contours and bounding blocks
    cv2.imshow("Image with Contours and Bounding Blocks", image_with_contours)
    cv2.waitKey(0)  # Wait indefinitely until a key is pressed
    cv2.destroyAllWindows()  # Close all OpenCV windows

In [258]:
image_path = "Desktop/Train_Samples_V0/Train_Samples_V0/20240328_153456.jpg"
display_resized_grayscale_image_with_contours_and_bounding_blocks(image_path, width=600, height=900)  # Specify width and height for resizing

Error: Unable to compute threshold. Image may not contain any white pixels.
Error: Threshold value is None. Cannot extract bounding blocks.


**THINNER AND GREEN CONTOUR RED BOX**

In [107]:
def display_resized_grayscale_image_with_contours_and_bounding_blocks(image_path, width=None, height=None):
    import os
    import cv2
    import numpy as np
    
    # Check if the file exists
    if not os.path.exists(image_path):
        print("Error: File does not exist at", image_path)
        return

    # Check if the file is readable
    if not os.access(image_path, os.R_OK):
        print("Error: Cannot read file at", image_path)
        return

    # Read the image from the file path
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Unable to load image from", image_path)
        return

    # Resize the image if width or height is provided
    if width is not None and height is not None:
        image = cv2.resize(image, (width, height))

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply smaller convolution to preserve finer details
    kernel = np.ones((3, 3), np.float32) / 50
    smoothed_image = cv2.filter2D(gray_image, -1, kernel)

    # Apply adaptive thresholding
    binary_image = cv2.adaptiveThreshold(smoothed_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank canvas for drawing the polygon boundaries and bounding boxes
    canvas = np.zeros_like(image)

    # Draw contours and bounding boxes on the canvas
    for contour in contours:
        # Draw contours with reduced thickness
        cv2.drawContours(canvas, [contour], -1, (0, 255, 0), 1)  # Thickness set to 1
        # Get bounding box coordinates
        x, y, w, h = cv2.boundingRect(contour)
        # Draw bounding boxes
        cv2.rectangle(canvas, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Display the canvas with polygon boundaries and bounding boxes
    cv2.imshow("Contours and Bounding Boxes", canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [108]:
image_path = "Desktop/Train_Samples_V0/Train_Samples_V0/20240328_153456.jpg"
display_resized_grayscale_image_with_contours_and_bounding_blocks(image_path, width=600, height=900)

**THINNER AND GREEN CONTOUR NO BOX**

In [123]:
def display_resized_grayscale_image_with_contours_and_bounding_blocks(image_path, width=None, height=None):
    
    
    # Check if the file exists
    if not os.path.exists(image_path):
        print("Error: File does not exist at", image_path)
        return

    # Check if the file is readable
    if not os.access(image_path, os.R_OK):
        print("Error: Cannot read file at", image_path)
        return

    # Read the image from the file path
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Unable to load image from", image_path)
        return

    # Resize the image if width or height is provided
    if width is not None and height is not None:
        image = cv2.resize(image, (width, height))

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply smaller convolution to preserve finer details
    kernel = np.ones((3, 3), np.float32) / 50
    smoothed_image = cv2.filter2D(gray_image, -1, kernel)

    # Apply adaptive thresholding
    binary_image = cv2.adaptiveThreshold(smoothed_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a blank canvas for drawing the polygon boundaries
    canvas = np.zeros_like(image)

    # Draw contours on the canvas
    for contour in contours:
        # Draw contours with reduced thickness
        cv2.drawContours(canvas, [contour], -1, (0, 255, 0), 1)  # Thickness set to 1

    # Display the canvas with polygon boundaries
    cv2.imshow("Contours", canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [124]:
image_path = "Desktop/Train_Samples_V0/Train_Samples_V0/20240328_153456.jpg"
display_resized_grayscale_image_with_contours_and_bounding_blocks(image_path, width=600, height=900)

**NOT A GOOD APPROACH COMPUTATIONALLY COSTLY**

In [182]:
def draw_edges_on_grid(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Define the grid parameters
    rows = 400
    cols = 600
    height, width = gray_image.shape

    # Calculate grid spacing
    grid_spacing_x = width // (cols + 1)
    grid_spacing_y = height // (rows + 1)

    # Define the Sobel kernel for edge detection
    sobel_kernel_x = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])

    # Iterate over each grid cell
    for y in range(grid_spacing_y, height, grid_spacing_y):
        for x in range(grid_spacing_x, width, grid_spacing_x):
            # Extract the current grid cell
            cell_image = gray_image[y-grid_spacing_y:y, x-grid_spacing_x:x]

            # Apply Sobel operator for edge detection
            edges_x = cv2.filter2D(cell_image, -1, sobel_kernel_x)

            # Find the position of leftmost and rightmost edges
            left_edge_x = np.argmax(edges_x, axis=1).min()
            right_edge_x = np.argmax(edges_x, axis=1).max()

            # Draw red lines indicating leftmost and rightmost edges
            left_edge_start = (x-grid_spacing_x+left_edge_x, y)
            left_edge_end = (x-grid_spacing_x+left_edge_x, y-grid_spacing_y)
            right_edge_start = (x-grid_spacing_x+right_edge_x, y)
            right_edge_end = (x-grid_spacing_x+right_edge_x, y-grid_spacing_y)

            # Check the shades of pixels on both sides of the red lines
            left_shade = gray_image[left_edge_end[1], left_edge_end[0]]
            right_shade = gray_image[right_edge_end[1], right_edge_end[0]]

            # Check conditions to remove or keep the red lines
            if left_shade > 150 and right_shade > 150:
                # Both sides are light, remove the red line
                continue
            elif left_shade < 100 and right_shade < 100:
                # Both sides are dark, remove the red line
                continue
            else:
                # At least one side is dark and the other side is light, keep the red line
                cv2.line(image, left_edge_start, left_edge_end, (0, 0, 255), 1)  # Red color
                cv2.line(image, right_edge_start, right_edge_end, (0, 0, 255), 1)  # Red color

    # Draw blue grids
    for y in range(grid_spacing_y, height, grid_spacing_y):
        cv2.line(image, (0, y), (width, y), (255, 0, 0), 1)
    for x in range(grid_spacing_x, width, grid_spacing_x):
        cv2.line(image, (x, 0), (x, height), (255, 0, 0), 1)

    # Display the image with red edges on blue grids
    resized_image = cv2.resize(image, (600, 900))
    cv2.imshow("Image with Red Edges on Blue Grids", resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [183]:
image_path = "Desktop/Train_Samples_V0/Train_Samples_V0/20240328_153456.jpg"
draw_edges_on_grid(image_path)

**IMPROVING THE BEST ONE**

In [265]:
def perform_ocr_with_edge_detection(image_path, merge_threshold=50):
    # Read the input image
    image = cv2.imread(image_path)

    # Resize the image to 600x900 pixels
    image = cv2.resize(image, (600, 900))

    # Step 1: Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Define the Sobel kernel for edge detection
    sobel_kernel_x = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])

    sobel_kernel_y = np.array([[-1, -2, -1],
                               [0, 0, 0],
                               [1, 2, 1]])

    # Step 3: Perform convolution for horizontal and vertical edges
    edges_x = cv2.filter2D(gray, -1, sobel_kernel_x)
    edges_y = cv2.filter2D(gray, -1, sobel_kernel_y)

    # Step 4: Combine horizontal and vertical edges
    edges = cv2.addWeighted(edges_x, 0.5, edges_y, 0.5, 0)

    # Step 5: Threshold the edge detection result
    _, thresholded = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY)

    # Step 6: Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Merge nearby bounding boxes
    merged_contours = merge_contours(contours, merge_threshold)

    # Step 7: Draw bounding boxes around detected contours
    for contour in merged_contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the result
    cv2.imshow('OCR Result', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def merge_contours(contours, merge_threshold):
    merged_contours = []
    for contour in contours:
        if merged_contours:
            merged = False
            for i, merged_contour in enumerate(merged_contours):
                x, y, w, h = cv2.boundingRect(contour)
                x_merged, y_merged, w_merged, h_merged = cv2.boundingRect(merged_contour)
                distance = np.sqrt((x - x_merged)**2 + (y - y_merged)**2)
                if distance < merge_threshold:
                    # Merge the contours
                    merged_contours[i] = np.vstack((merged_contour, contour))
                    merged = True
                    break
            if not merged:
                merged_contours.append(contour)
        else:
            merged_contours.append(contour)
    return merged_contours

In [266]:
image_path = 'Desktop/Train_Samples_V0/Train_Samples_V0/20240328_153456.jpg'
perform_ocr_with_edge_detection(image_path, merge_threshold=10)


**MERGED+SMOOTHENED**

In [262]:
def improved_perform_ocr_with_edge_detection(image_path, merge_threshold=50, smoothing_radius=10):
    # Read the input image
    image = cv2.imread(image_path)

    # Resize the image to 600x900 pixels
    image = cv2.resize(image, (600, 900))

    # Step 1: Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Define the Sobel kernel for edge detection
    sobel_kernel_x = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])

    sobel_kernel_y = np.array([[-1, -2, -1],
                               [0, 0, 0],
                               [1, 2, 1]])

    # Step 3: Perform convolution for horizontal and vertical edges
    edges_x = cv2.filter2D(gray, -1, sobel_kernel_x)
    edges_y = cv2.filter2D(gray, -1, sobel_kernel_y)

    # Step 4: Combine horizontal and vertical edges
    edges = cv2.addWeighted(edges_x, 0.5, edges_y, 0.5, 0)

    # Step 5: Threshold the edge detection result
    _, thresholded = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY)

    # Step 6: Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Merge nearby bounding boxes
    merged_contours = merge_contours(contours, merge_threshold)

    # Smooth outer boxes
    smoothed_boxes = smooth_outer_boxes(merged_contours, smoothing_radius)

    # Step 7: Draw bounding boxes around detected contours
    for box in smoothed_boxes:
        x, y, w, h = cv2.boundingRect(box)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the result
    cv2.imshow('OCR Result', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def merge_contours(contours, merge_threshold):
    merged_contours = []
    for contour in contours:
        if merged_contours:
            merged = False
            for i, merged_contour in enumerate(merged_contours):
                x, y, w, h = cv2.boundingRect(contour)
                x_merged, y_merged, w_merged, h_merged = cv2.boundingRect(merged_contour)
                distance = np.sqrt((x - x_merged)**2 + (y - y_merged)**2)
                if distance < merge_threshold:
                    # Merge the contours
                    merged_contours[i] = np.vstack((merged_contour, contour))
                    merged = True
                    break
            if not merged:
                merged_contours.append(contour)
        else:
            merged_contours.append(contour)
    return merged_contours

def smooth_outer_boxes(contours, smoothing_radius):
    smoothed_boxes = []
    for contour in contours:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        smoothed_boxes.append(cv2.convexHull(approx))
    return smoothed_boxes

In [263]:
image_path = 'Desktop/Train_Samples_V0/Train_Samples_V0/20240328_153456.jpg'
improved_perform_ocr_with_edge_detection(image_path, merge_threshold=15, smoothing_radius=20)

In [264]:
image_path = 'Desktop/Train_Samples_V0/Train_Samples_V0/20240328_154830.jpg'
improved_perform_ocr_with_edge_detection(image_path, merge_threshold=16, smoothing_radius=20)

**ADDING ROWS SEPERATION**

In [270]:
def improved_perform_ocr_with_edge_detection(image_path, merge_threshold=16, smoothing_radius=20):
    # Read the input image
    image = cv2.imread(image_path)

    # Resize the image to 600x900 pixels
    image = cv2.resize(image, (600, 900))

    # Add uniformly spaced horizontal lines
    num_lines = 10
    height, width, _ = image.shape
    spacing = height // (num_lines + 1)
    for i in range(1, num_lines + 1):
        y = i * spacing
        cv2.line(image, (0, y), (width, y), (0, 0, 255), 1)  # Red lines with thickness 1

    # Step 1: Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Define the Sobel kernel for edge detection
    sobel_kernel_x = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])

    sobel_kernel_y = np.array([[-1, -2, -1],
                               [0, 0, 0],
                               [1, 2, 1]])

    # Step 3: Perform convolution for horizontal and vertical edges
    edges_x = cv2.filter2D(gray, -1, sobel_kernel_x)
    edges_y = cv2.filter2D(gray, -1, sobel_kernel_y)

    # Step 4: Combine horizontal and vertical edges
    edges = cv2.addWeighted(edges_x, 0.5, edges_y, 0.5, 0)

    # Step 5: Threshold the edge detection result
    _, thresholded = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY)

    # Step 6: Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Merge nearby bounding boxes
    merged_contours = merge_contours(contours, merge_threshold)

    # Smooth outer boxes
    smoothed_boxes = smooth_outer_boxes(merged_contours, smoothing_radius)

    # Step 7: Draw bounding boxes around detected contours
    for box in smoothed_boxes:
        x, y, w, h = cv2.boundingRect(box)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the result
    cv2.imshow('OCR Result', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def merge_contours(contours, merge_threshold):
    merged_contours = []
    for contour in contours:
        if merged_contours:
            merged = False
            for i, merged_contour in enumerate(merged_contours):
                x, y, w, h = cv2.boundingRect(contour)
                x_merged, y_merged, w_merged, h_merged = cv2.boundingRect(merged_contour)
                distance = np.sqrt((x - x_merged)**2 + (y - y_merged)**2)
                if distance < merge_threshold:
                    # Merge the contours
                    merged_contours[i] = np.vstack((merged_contour, contour))
                    merged = True
                    break
            if not merged:
                merged_contours.append(contour)
        else:
            merged_contours.append(contour)
    return merged_contours

def smooth_outer_boxes(contours, smoothing_radius):
    smoothed_boxes = []
    for contour in contours:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        smoothed_boxes.append(cv2.convexHull(approx))
    return smoothed_boxes


In [271]:
image_path = 'Desktop/Train_Samples_V0/Train_Samples_V0/20240328_154830.jpg'
improved_perform_ocr_with_edge_detection(image_path)

**RUNNING ON THE DIR**

**FIRST DIR**

In [272]:
from tqdm import tqdm

In [276]:

def improved_perform_ocr_with_edge_detection(image_path, merge_threshold=16, smoothing_radius=20):
    # Read the input image
    image = cv2.imread(image_path)

    # Resize the image to 600x900 pixels
    image = cv2.resize(image, (600, 900))

    # Add uniformly spaced horizontal lines
    num_lines = 10
    height, width, _ = image.shape
    spacing = height // (num_lines + 1)
    for i in range(1, num_lines + 1):
        y = i * spacing
        cv2.line(image, (0, y), (width, y), (0, 0, 255), 1)  # Red lines with thickness 1

    # Step 1: Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Define the Sobel kernel for edge detection
    sobel_kernel_x = np.array([[-1, 0, 1],
                               [-2, 0, 2],
                               [-1, 0, 1]])

    sobel_kernel_y = np.array([[-1, -2, -1],
                               [0, 0, 0],
                               [1, 2, 1]])

    # Step 3: Perform convolution for horizontal and vertical edges
    edges_x = cv2.filter2D(gray, -1, sobel_kernel_x)
    edges_y = cv2.filter2D(gray, -1, sobel_kernel_y)

    # Step 4: Combine horizontal and vertical edges
    edges = cv2.addWeighted(edges_x, 0.5, edges_y, 0.5, 0)

    # Step 5: Threshold the edge detection result
    _, thresholded = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY)

    # Step 6: Find contours in the thresholded image
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Merge nearby bounding boxes
    merged_contours = merge_contours(contours, merge_threshold)

    # Smooth outer boxes
    smoothed_boxes = smooth_outer_boxes(merged_contours, smoothing_radius)

    # Step 7: Draw bounding boxes around detected contours
    for box in smoothed_boxes:
        x, y, w, h = cv2.boundingRect(box)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Save the result image with bounding boxes
    result_path = os.path.join(os.path.expanduser("~"), 'Desktop', 'result_images', os.path.basename(image_path))
    cv2.imwrite(result_path, image)

def merge_contours(contours, merge_threshold):
    merged_contours = []
    for contour in contours:
        if merged_contours:
            merged = False
            for i, merged_contour in enumerate(merged_contours):
                x, y, w, h = cv2.boundingRect(contour)
                x_merged, y_merged, w_merged, h_merged = cv2.boundingRect(merged_contour)
                distance = np.sqrt((x - x_merged)**2 + (y - y_merged)**2)
                if distance < merge_threshold:
                    # Merge the contours
                    merged_contours[i] = np.vstack((merged_contour, contour))
                    merged = True
                    break
            if not merged:
                merged_contours.append(contour)
        else:
            merged_contours.append(contour)
    return merged_contours

def smooth_outer_boxes(contours, smoothing_radius):
    smoothed_boxes = []
    for contour in contours:
        epsilon = 0.01 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        smoothed_boxes.append(cv2.convexHull(approx))
    return smoothed_boxes

# Create the result_images directory if it doesn't exist on the desktop
result_images_dir = os.path.join(os.path.expanduser("~"), 'Desktop', 'result_images')
if not os.path.exists(result_images_dir):
    os.makedirs(result_images_dir)

# Process all images in a folder
image_folder = 'Desktop/Train_Samples_V0/Train_Samples_V0/'
for filename in tqdm(os.listdir(image_folder)):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_folder, filename)
        improved_perform_ocr_with_edge_detection(image_path)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [00:53<00:00,  1.92s/it]


**SECOND DIR**

In [286]:
def compute_threshold(image):
    # Compute MWR and AWR for each line
    mwr_values = []
    awr_values = []
    for row in image:
        # Count white run-lengths
        white_runs = [len(run) for run in np.split(row, np.where(np.diff(row) != 0)[0] + 1) if run.size > 0 and run[0] == 255]
        if white_runs:
            mwr_values.append(np.median(white_runs))
            awr_values.append(len(row) / (len(white_runs) + 1))  # +1 to avoid division by zero
    # Check if there are values in mwr_values and awr_values
    if mwr_values and awr_values:
        # Compute threshold using MWR and AWR
        threshold = np.median(mwr_values) * np.median(awr_values)
        return threshold
    else:
        print("Error: Unable to compute threshold. Image may not contain any white pixels.")
        return None

In [290]:
def extract_bounding_blocks(image, contours, threshold, padding=20):
    if threshold is None:
        print("Error: Threshold value is None. Cannot extract bounding blocks.")
        return []

    bounding_blocks = []
    for contour in contours:
        # Get bounding box of contour
        x, y, w, h = cv2.boundingRect(contour)
        # Apply padding
        x -= padding
        y -= padding
        w += 2 * padding
        h += 2 * padding
        # Check if width is greater than threshold
        if w > threshold:
            # Merge with existing bounding boxes if overlapping
            merged = False
            for i, (bx, by, bw, bh) in enumerate(bounding_blocks):
                if (x >= bx and x <= bx + bw) or (x + w >= bx and x + w <= bx + bw) or \
                   (bx >= x and bx <= x + w) or (bx + bw >= x and bx + bw <= x + w):
                    # Overlapping, merge boxes
                    x = min(x, bx)
                    y = min(y, by)
                    w = max(x + w, bx + bw) - x
                    h = max(y + h, by + bh) - y
                    bounding_blocks[i] = (x, y, w, h)
                    merged = True
                    break
            if not merged:
                bounding_blocks.append((x, y, w, h))
    return bounding_blocks

In [ ]:
def display_resized_grayscale_image_with_contours_and_bounding_blocks(image_path, output_dir, width=None, height=None):
    # Check if the file exists
    if not os.path.exists(image_path):
        print("Error: File does not exist at", image_path)
        return

    # Check if the file is readable
    if not os.access(image_path, os.R_OK):
        print("Error: Cannot read file at", image_path)
        return

    # Read the image from the file path
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Unable to load image from", image_path)
        return

    # Resize the image if width or height is provided
    if width is not None and height is not None:
        image = cv2.resize(image, (width, height))

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply convolution for edge detection (you can use other methods too)
    edges = cv2.Canny(gray_image, 100, 200)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Compute threshold
    threshold = compute_threshold(gray_image)

    # Extract bounding blocks based on contours and threshold
    bounding_blocks = extract_bounding_blocks(gray_image, contours, threshold)

    # Draw contours on the original image
    image_with_contours = image.copy()
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)  # Green color for contours

    # Draw bounding blocks
    for x, y, w, h in bounding_blocks:
        cv2.rectangle(image_with_contours, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color for bounding blocks

    # Save the output image
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, image_with_contours)

# Directory containing input images
input_dir = r"Desktop\Train_Samples_V0\Train_Samples_V0"

# Directory to save output images
output_dir = r"Desktop\result_image_2"
os.makedirs(output_dir, exist_ok=True)

# Loop through input images with tqdm progress bar
for filename in tqdm(os.listdir(input_dir)):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        input_image_path = os.path.join(input_dir, filename)
        display_resized_grayscale_image_with_contours_and_bounding_blocks(input_image_path, output_dir, width=800, height=600)

**DIR 3**

In [ ]:
def display_final_resized_grayscale_image_with_contours_and_bounding_blocks(image_path, output_dir, width=None, height=None):
    # Check if the file exists
    if not os.path.exists(image_path):
        print("Error: File does not exist at", image_path)
        return

    # Check if the file is readable
    if not os.access(image_path, os.R_OK):
        print("Error: Cannot read file at", image_path)
        return

    # Read the image from the file path
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Unable to load image from", image_path)
        return

    # Resize the image if width or height is provided
    if width is not None and height is not None:
        image = cv2.resize(image, (width, height))

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply convolution for edge detection (you can use other methods too)
    edges = cv2.Canny(gray_image, 100, 200)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Compute threshold
    threshold = compute_threshold(gray_image)

    # Extract bounding blocks based on contours and threshold
    bounding_blocks = extract_bounding_blocks(gray_image, contours, threshold)

    # Draw contours on the original image
    image_with_contours = image.copy()
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 1)  # Green color for contours

    # Draw bounding blocks
    for x, y, w, h in bounding_blocks:
        cv2.rectangle(image_with_contours, (x, y), (x + w, y + h), (0, 0, 255), 1)  # Red color for bounding blocks

    # Save the output image
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, image_with_contours)

# Directory containing input images
input_dir = r"Desktop\Train_Samples_V0\Train_Samples_V0"

# Directory to save output images
output_dir = r"Desktop\result_image_3"
os.makedirs(output_dir, exist_ok=True)

# Loop through input images with tqdm progress bar
for filename in tqdm(os.listdir(input_dir)):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        input_image_path = os.path.join(input_dir, filename)
        display_resized_grayscale_image_with_contours_and_bounding_blocks(input_image_path, output_dir, width=800, height=600)

In [ ]:
def display_resized_grayscale_image_with_contours_and_bounding_blocks(image_path, output_dir, width=None, height=None):
    # Check if the file exists
    if not os.path.exists(image_path):
        print("Error: File does not exist at", image_path)
        return

    # Check if the file is readable
    if not os.access(image_path, os.R_OK):
        print("Error: Cannot read file at", image_path)
        return

    # Read the image from the file path
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Unable to load image from", image_path)
        return

    # Resize the image if width or height is provided
    if width is not None and height is not None:
        image = cv2.resize(image, (width, height))

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply convolution for edge detection (you can use other methods too)
    edges = cv2.Canny(gray_image, 100, 200)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Compute threshold
    threshold = compute_threshold(gray_image)

    # Extract bounding blocks based on contours and threshold
    bounding_blocks = extract_bounding_blocks(gray_image, contours, threshold)

    # Draw contours on the original image
    image_with_contours = image.copy()
    cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 2)  # Green color for contours

    # Draw bounding blocks
    for x, y, w, h in bounding_blocks:
        cv2.rectangle(image_with_contours, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red color for bounding blocks

    # Save the output image
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, image_with_contours)

# Directory containing input images
input_dir = r"Desktop\Train_Samples_V0\Train_Samples_V0"

# Directory to save output images
output_dir = r"Desktop\result_image_2"
os.makedirs(output_dir, exist_ok=True)

# Loop through input images with tqdm progress bar
for filename in tqdm(os.listdir(input_dir)):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        input_image_path = os.path.join(input_dir, filename)
        display_resized_grayscale_image_with_contours_and_bounding_blocks(input_image_path, output_dir, width=800, height=600)

**APPLYING BOTH THE BEST**

In [ ]:
def compute_threshold(image):
    # Compute MWR and AWR for each line
    mwr_values = []
    awr_values = []
    for row in image:
        # Count white run-lengths
        white_runs = [len(run) for run in np.split(row, np.where(np.diff(row) != 0)[0] + 1) if run.size > 0 and run[0] == 255]
        if white_runs:
            mwr_values.append(np.median(white_runs))
            awr_values.append(len(row) / (len(white_runs) + 1))  # +1 to avoid division by zero
    # Check if there are values in mwr_values and awr_values
    if mwr_values and awr_values:
        # Compute threshold using MWR and AWR
        threshold = np.median(mwr_values) * np.median(awr_values)
        return threshold
    else:
        print("Error: Unable to compute threshold. Image may not contain any white pixels.")
        return None

In [ ]:
def extract_bounding_blocks(image, contours, threshold, padding=20):
    if threshold is None:
        print("Error: Threshold value is None. Cannot extract bounding blocks.")
        return []

    bounding_blocks = []
    for contour in contours:
        # Get bounding box of contour
        x, y, w, h = cv2.boundingRect(contour)
        # Apply padding
        x -= padding
        y -= padding
        w += 2 * padding
        h += 2 * padding
        # Check if width is greater than threshold
        if w > threshold:
            # Merge with existing bounding boxes if overlapping
            merged = False
            for i, (bx, by, bw, bh) in enumerate(bounding_blocks):
                if (x >= bx and x <= bx + bw) or (x + w >= bx and x + w <= bx + bw) or \
                   (bx >= x and bx <= x + w) or (bx + bw >= x and bx + bw <= x + w):
                    # Overlapping, merge boxes
                    x = min(x, bx)
                    y = min(y, by)
                    w = max(x + w, bx + bw) - x
                    h = max(y + h, by + bh) - y
                    bounding_blocks[i] = (x, y, w, h)
                    merged = True
                    break
            if not merged:
                bounding_blocks.append((x, y, w, h))
    return bounding_blocks

In [254]:
image_path = 'Desktop/Train_Samples_V0/Train_Samples_V0/20240328_154830.jpg'
display_final_image(image_path)